# Repertoire

In [ ]:
try:
  from google.colab import drive
  drive.mount('/gdrive')
except:
  print("vous n'êtes encore  sur Google Colab")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [ ]:
REP_PROJET = '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
REP_INPUT  = REP_PROJET + 'Input/'
REP_INTERMED  = REP_PROJET + 'Intermed/'
REP_OUTPUT  = REP_PROJET + 'Output/'

# MODULES

In [ ]:
%cd '/gdrive/My Drive/Colab Notebooks/Dedoublement/'
!pwd

/gdrive/My Drive/Colab Notebooks/Dedoublement
/gdrive/My Drive/Colab Notebooks/Dedoublement


In [ ]:
%%capture
pip install -r requirements.txt

In [ ]:
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
import numpy as np

# FUNCTIONS

In [ ]:
%run "00_functions.ipynb"

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Discogs data

## Load 

In [ ]:
baseline = pd.read_pickle(REP_INTERMED+ "baseline.sav")

## Vectorizer :  Keep TF-IDF strategy 

In [ ]:
start_time = datetime.now()

create_vectorizer_and_vectorized_db(baseline, analyzer = 'char',ngram_range=(2,2))

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

# Duration: 02 min:15s

**** creation du vectorizer


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


nb features: 3570
**** sauvegarde des fichiers en pickle pour la prochaine fois
Duration: 0:03:44.347460


In [ ]:
analyzer = 'char'
ngram_range = (2,2)
X_train, vectorizer = reload_data_and_vectorizer(analyzer = analyzer, ngram_range=ngram_range)

# Query Data

In [ ]:
df_no_match = pd.read_pickle(REP_INTERMED +'base_cdandlp.sav').reset_index(drop=True)

df_no_match['text'] = df_no_match['artiste'] +  ' ' + df_no_match['titre']

df_sample = df_no_match.sample(100, random_state=20).reset_index(drop=True)

del df_no_match

# KNN unsupervised (sklearn api)

## Training with Manahattan

In [ ]:

suffix_gram = re.sub(r"[(,)]", "", str(ngram_range)).replace(' ','_')

path = REP_INTERMED + analyzer + '_' + suffix_gram+  '_results_sim_Manhattan.pkl'

nbrs = NearestNeighbors(n_neighbors=100, metric="manhattan",algorithm='brute').fit(X_train)

start_time = datetime.now()

result = [ nbrs.kneighbors(text_to_vect(df_sample.iloc[i,3], vectorizer)) for i in range (len(df_sample)) ]

f = open(path, 'wb')

pickle.dump(result, f) 

f.close()

end_time = datetime.now()

print('Duration: {}'.format(end_time - start_time))

Duration: 0:04:48.443891


# Customiz Output

In [ ]:
suffix_gram = re.sub(r"[(,)]", "", str(ngram_range)).replace(' ','_')
analyzer = 'char'
ngram_range = (2,2)

path = REP_INTERMED + analyzer + '_' + suffix_gram+  '_results_sim_Manhattan.pkl'

with open(path, 'rb') as file:
      result = pickle.load(file)

In [ ]:
temp = [ customize_output(i,results= result, df_baseline=baseline.copy(), sample=df_sample) for i in range(len(df_sample)) ]

In [ ]:
pd.concat(temp,axis=0).\
    rename(columns={'master_id':'id_text'}).\
    to_excel(REP_INTERMED + analyzer + '_' + suffix_gram +'_neighbors_Manhatthan.xlsx', index= False)

In [ ]:
pd.concat(temp,axis=0)

,id_query,master_id,artist,title,distance,rank,levenshtein,jaro_winkler,jaccard,overlap,hamming,fuzzy_partial
0,114852181,114852181,tiffany,i think we're alone now,-1.0,0,1,1,1,1,1,1
1,114852181,"314885,74319",Tiffany,I Think We'Re Alone Now,1.829447,1,1.0,1.0,1.0,1.0,1.0,1.0
2,114852181,314885,Tiffany,I Think We’Re Alone Now,1.829447,1,1.0,1.0,1.0,1.0,1.0,1.0
3,114852181,74319,Tiffany,I Think We'Re Alone Now (12'' Remix),3.284481,3,0.869565,0.953846,0.769231,1.0,0.769231,1.0
4,114852181,0,Tiffany,I Think We'Re Alone Now / Hey Baby,3.392809,4,0.869565,0.953846,0.769231,1.0,0.769231,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
96,119859709,0,/ / / /,A,5.515528,3,0.043478,0.0,0.022222,1.0,0.0,1.0
97,119859709,0,.......,.​.​.​.​.​.​.,5.515528,3,0.0,0.0,0.0,0.0,0.0,0.0
98,119859709,0,. (4),/​/​/​///,5.515528,3,0.0,0.0,0.0,0.0,0.0,0.0
99,119859709,0,>,>,5.515528,3,0.0,0.0,0.0,0.0,0.0,0.0
